In [1]:
Text = "I am learning NLP" 

In [2]:
import pandas as pd
pd.get_dummies(Text.split())

,I,NLP,am,learning
0,True,False,False,False
1,False,False,True,False
2,False,False,False,True
3,False,True,False,False


In [3]:
text = ["i love NLP and i will learn NLP in 2month"]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer= CountVectorizer()
vectorizer.fit(text)
vector= vectorizer.transform(text)

In [6]:
print(vectorizer.vocabulary_)
print(vector.toarray())

{'love': 4, 'nlp': 5, 'and': 1, 'will': 6, 'learn': 3, 'in': 2, '2month': 0}
[[1 1 1 1 1 2 1]]


In [8]:
df= pd.DataFrame(data=vector.toarray(),columns=vectorizer.get_feature_names_out())
df

,2month,and,in,learn,love,nlp,will
0,1,1,1,1,1,2,1


In [9]:
text= 'I am learning NLP'

In [10]:
from textblob import TextBlob
TextBlob(text).ngrams(1)

[WordList(['I']), WordList(['am']), WordList(['learning']), WordList(['NLP'])]

In [13]:
TextBlob(text).ngrams(2)

[WordList(['I', 'am']),
 WordList(['am', 'learning']),
 WordList(['learning', 'NLP'])]

In [14]:
TextBlob(text).ngrams(3)

[WordList(['I', 'am', 'learning']), WordList(['am', 'learning', 'NLP'])]

In [15]:
TextBlob(text).ngrams(4)

[WordList(['I', 'am', 'learning', 'NLP'])]

In [16]:
# import libraries
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [17]:
df= pd.read_csv(r"C:\Users\kesha\anaconda3\covid_fake.csv")

In [18]:
df.head()

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake


In [19]:
df.shape

(1164, 4)

In [21]:
df['label'].value_counts()

label
TRUE    584
Fake    345
fake    230
Name: count, dtype: int64

In [22]:
df.loc[5:15]

,title,text,source,label
5,CORONA UNMASKED: Chinese Intelligence Officer ...,NaN,NaN,NaN
6,NaN,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,Fake
7,NaN,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,Fake
8,NaN,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,Fake
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,TRUE
10,NaN,The new Coronavirus may not show signs of infe...,Taiwan Experts,Fake
11,NaN,A vaccine meant for cattle can be used to figh...,facebook,Fake
12,NaN,Using a hair dryer to breathe in hot air can c...,Youtube,Fake
13,NaN,Corona virus before it reaches the lungs it re...,twitter,Fake
14,Exposing yourself to the sun or to temperature...,"You can catch COVID-19, no matter how sunny or...",https://www.who.int/emergencies/diseases/novel...,TRUE
